In [14]:
import os
import json
from azure.ai.evaluation import (
    evaluate, 
    IntentResolutionEvaluator,
    ToolCallAccuracyEvaluator,
    TaskAdherenceEvaluator,
    AzureOpenAIModelConfiguration,
)
from pprint import pprint
from dotenv import load_dotenv

load_dotenv('../.env')

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)
file_path = "evaluation_dataset.jsonl"

In [15]:
tool_definitions=[
    {
        "name": "BookingPlugin-check_availability",
        "description": "指定された日付で客室が利用可能かどうかを確認します。",
        "parameters": {
            "type": "object",
            "properties": {
                "room_type": {
                    "type": "string",
                    "description": "客室タイプ。"
                },
                "date": {
                    "type": "string",
                    "description": "予約日（YYYY-MM-DD形式）。"
                }
            }
        }
    },
    {
        "name": "BookingPlugin-confirm_booking",
        "description": "予約を確定し、客室数を減らします。",
        "parameters": {
            "type": "object",
            "properties": {
                "room_type": {
                    "type": "string",
                    "description": "客室タイプ。"
                },
                "date": {
                    "type": "string",
                    "description": "予約日（YYYY-MM-DD形式）。"
                },
                "count": {
                    "type": "integer",
                    "description": "予約する客室数。"
                }
            }
        }
    },
    {
        "name": "DiningPlugin-get_specials",
        "description": "本日のダイニングスペシャルを提供します。",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "DiningPlugin-get_item_price",
        "description": "指定されたメニュー項目の価格を提供します。",
        "parameters": {
            "type": "object",
            "properties": {
                "menu_item": {
                    "type": "string",
                    "description": "メニュー項目名。"
                }
            }
        }
    },
    {
        "name": "DiningPlugin-reserve_table",
        "description": "ホテルレストランでのテーブル予約をシミュレートします。",
        "parameters": {
            "type": "object",
            "properties": {
                "time": {
                    "type": "string",
                    "description": "予約時間（例：HH:MM）。"
                },
                "party_size": {
                    "type": "integer",
                    "description": "予約人数。"
                }
            }
        }
    },
    {
        "name": "SemanticSearchPlugin-search_rooms_by_description",
        "description": "客室の説明に基づいてセマンティック検索でホテルの客室を検索します。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "ユーザーが探している客室タイプの説明。"
                }
            }
        }
    },
    {
        "name": "TimePlugin-get_today",
        "description": "今日の日付をYYYY-MM-DD形式で返します。",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "TimePlugin-get_relative_date",
        "description": "日数のオフセットに基づいて相対的な日付を返します。",
        "parameters": {
            "type": "object",
            "properties": {
                "days_offset": {
                    "type": "integer",
                    "description": "今日に追加する日数。"
                }
            }
        }
    }
]

In [16]:
intent_resolution_evaluator = IntentResolutionEvaluator(model_config=model_config)

# JSONLファイル全体を開いて読み込み
print("意図解決評価")
print("--------------------------------------------------")
with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        entry = json.loads(line)
        query = entry.get("query", "")
        response = entry.get("response", "")

        print(f"--- エントリ {i} ---")
        print(f"クエリ           : {query}")
        print(f"レスポンス        : {response}")
        print(f"ツール定義 : {tool_definitions}")
        result = intent_resolution_evaluator(
            query=query,
            response=response,
            tool_definitions=tool_definitions,
        )
        pprint(f"意図解決結果: {result}")
        print("--------------------------------------------------")

In [17]:
query = "明日のデラックスルームの空室状況を確認してください。"
file_path = "evaluation_dataset.jsonl"

tool_calls=[
    {
        "type": "tool_call",
        "tool_call_id": "call_1744299539059",
        "name": "TimePlugin-get_relative_date",
        "arguments": {
            "days_offset": 1
        },
    },
    {
        "type": "tool_call",
        "tool_call_id": "call_1744299539059",
        "name": "BookingPlugin-check_availability",
        "arguments": "{\"room_type\":\"deluxe\",\"date\":\"2025-04-11\"}"
    }
]

tool_definitions=[
    {
        "name": "BookingPlugin-check_availability",
        "description": "指定された日付で客室が利用可能かどうかを確認します。",
        "parameters": {
            "type": "object",
            "properties": {
                "room_type": {
                    "type": "string",
                    "description": "客室タイプ。"
                },
                "date": {
                    "type": "string",
                    "description": "予約日（YYYY-MM-DD形式）。"
                }
            }
        }
    },
    {
        "name": "BookingPlugin-confirm_booking",
        "description": "予約を確定し、客室数を減らします。",
        "parameters": {
            "type": "object",
            "properties": {
                "room_type": {
                    "type": "string",
                    "description": "客室タイプ。"
                },
                "date": {
                    "type": "string",
                    "description": "予約日（YYYY-MM-DD形式）。"
                },
                "count": {
                    "type": "integer",
                    "description": "予約する客室数。"
                }
            }
        }
    },
    {
        "name": "DiningPlugin-get_specials",
        "description": "本日のダイニングスペシャルを提供します。",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "DiningPlugin-get_item_price",
        "description": "指定されたメニュー項目の価格を提供します。",
        "parameters": {
            "type": "object",
            "properties": {
                "menu_item": {
                    "type": "string",
                    "description": "メニュー項目名。"
                }
            }
        }
    },
    {
        "name": "DiningPlugin-reserve_table",
        "description": "ホテルレストランでのテーブル予約をシミュレートします。",
        "parameters": {
            "type": "object",
            "properties": {
                "time": {
                    "type": "string",
                    "description": "予約時間（例：HH:MM）。"
                },
                "party_size": {
                    "type": "integer",
                    "description": "予約人数。"
                }
            }
        }
    },
    {
        "name": "SemanticRoomSearchPlugin-search_rooms_by_description",
        "description": "客室の説明に基づいてセマンティック検索でホテルの客室を検索します。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "ユーザーが探している客室タイプの説明。"
                }
            }
        }
    },
    {
        "name": "TimePlugin-get_today",
        "description": "今日の日付をYYYY-MM-DD形式で返します。",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "TimePlugin-get_relative_date",
        "description": "日数のオフセットに基づいて相対的な日付を返します。",
        "parameters": {
            "type": "object",
            "properties": {
                "days_offset": {
                    "type": "integer",
                    "description": "今日に追加する日数。"
                }
            }
        }
    }
]
tool_call_accuracy_evaluator = ToolCallAccuracyEvaluator(model_config)

response = tool_call_accuracy_evaluator(
    query=query, 
    tool_calls=tool_calls, 
    tool_definitions=tool_definitions,
)
pprint(response)

In [18]:
tool_call_accuracy_evaluator = ToolCallAccuracyEvaluator(model_config=model_config)

with open(file_path, "r") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        record = json.loads(line)
        query = record.get("query", "クエリが提供されていません")
        tool_calls = record.get("tool_calls", [])
        response = record.get("response", "レスポンスが提供されていません")

        print(f"クエリ: {query}")
        print("\nツール呼び出し:")
        pprint(tool_calls)

        if tool_calls:  # tool_callsが空でない場合のみ評価
            metric = tool_call_accuracy_evaluator(
                query=query, 
                tool_calls=tool_calls, 
                tool_definitions=tool_definitions,
            )
            print("\nツール呼び出し精度結果: ")
            pprint(metric)
        else:
            print("\n評価するツール呼び出しがありません。")
        print("-----------------------------------------------")

In [19]:
task_adherence_evaluator = TaskAdherenceEvaluator(model_config=model_config)

with open(file_path, "r") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        record = json.loads(line)
        query = record.get("query", "クエリが提供されていません")
        response = record.get("response", "レスポンスが提供されていません")

        print(f"クエリ: {query}")
        print(f"レスポンス: {response}")

        metric = task_adherence_evaluator(
            query=query, 
            response=response, 
            tool_definitions=tool_definitions,
        )
        print("\nタスク遵守結果: ")
        pprint(metric)
        print("-----------------------------------------------")

In [ ]:
import random

agentic_evals = evaluate(
    data=file_path,
    evaluation_name=f"agentic_evals_{random.randint(1, 10000)}",
    evaluators={
        "intent_resolution": intent_resolution_evaluator,
        "tool_call_accuracy": tool_call_accuracy_evaluator,
        "task_adherence": task_adherence_evaluator,
    },
    evaluator_config={
        "tool_call_accuracy": {
            "query": "{data.query}",
            "tool_calls": "{data.tool_calls}",
        },
        "task_adherence": {
            "query": "{data.query}",
            "response": "{data.response}",
        },
        "intent_resolution": {
            "query": "{data.query}",
            "response": "{data.response}",
        },
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    }
)
print(agentic_evals)

意図解決評価
--------------------------------------------------
--- エントリ 1 ---
クエリ           : 海が見える部屋を探しています。検索してもらえますか？
レスポンス        : 海の景色をご覧いただける客室をご案内いたします：

🏨 **Suite**
📅 日付: 2025年6月5日
🛏 説明: キングサイズベッド、オーシャンビュー、エレガントな装飾を備えた広々とした豪華なスイートルーム。ロマンチックな休暇に最適です。
💵 料金: $250
🟢 空室数: 2 室

こちらのスイートルームにご興味がございましたら、または他にご不明な点がございましたら、お気軽にお申し付けください！
ツール定義 : [{'name': 'BookingPlugin-check_availability', 'description': '指定された日付で客室が利用可能かどうかを確認します。', 'parameters': {'type': 'object', 'properties': {'room_type': {'type': 'string', 'description': '客室タイプ。'}, 'date': {'type': 'string', 'description': '予約日（YYYY-MM-DD形式）。'}}}}, {'name': 'BookingPlugin-confirm_booking', 'description': '予約を確定し、客室数を減らします。', 'parameters': {'type': 'object', 'properties': {'room_type': {'type': 'string', 'description': '客室タイプ。'}, 'date': {'type': 'string', 'description': '予約日（YYYY-MM-DD形式）。'}, 'count': {'type': 'integer', 'description': '予約する客室数。'}}}}, {'name': 'DiningPlugin-get_specials', 'description': '本日のダイニングスペシャルを提供します。', 'param